# Gerekli paketlerin yüklenmesi

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 70 kB/s 
     |████████████████████████████████| 198 kB 44.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=f4c325f052bee89f358c87a95599b5af4c460a9d406b004a0340270c037e1e9d
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


# SparkSession oluşturma

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark").getOrCreate()

# Github clone ile diyabet veri setini COLAB'a aktarma

In [3]:
! git clone https://github.com/biostatisticsphd/diabetes_dataset

Cloning into 'diabetes_dataset'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 2), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), done.


In [4]:
! ls diabetes_dataset

diabetes.csv  new_test.csv  README.md


In [5]:
df = spark.read.csv('/content/diabetes_dataset/diabetes.csv',header=True,inferSchema=True)

In [6]:
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|      0|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|            0|            0|      0|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|      0|25.6|                   0.294| 28|      0|


In [7]:
df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [8]:
df.groupby('Outcome').count().show()

+-------+-----+
|Outcome|count|
+-------+-----+
|      1|  684|
|      0| 1316|
+-------+-----+



In [9]:
df.describe().show()

+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|summary|      Pregnancies|           Glucose|     BloodPressure|    SkinThickness|          Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------------+------------------+------------------+
|  count|             2000|              2000|              2000|             2000|             2000|              2000|                    2000|              2000|              2000|
|   mean|           3.7035|          121.1825|           69.1455|           20.935|           80.254|32.192999999999984|     0.47092999999999974|           33.0905|             0.342|
| stddev|3.306063032730656|32.068635649902916|19.188314815604098|16.103242909926

In [10]:
for col in df.columns:
  print(col+":",df[df[col].isNull()].count())

Pregnancies: 0
Glucose: 0
BloodPressure: 0
SkinThickness: 0
Insulin: 0
BMI: 0
DiabetesPedigreeFunction: 0
Age: 0
Outcome: 0


In [11]:
def count_zeros():
  columns_list =['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
  for i in columns_list:
    print(i+":",df[df[i]==0].count())

In [12]:
count_zeros()

Glucose: 13
BloodPressure: 90
SkinThickness: 573
Insulin: 956
BMI: 28


In [13]:
from pyspark.sql.functions import *
for i in df.columns[1:6]:
  data = df.agg({i:'mean'}).first()[0]
  print("Mean value for {} is {}".format(i,int(data)))
  df = df.withColumn(i,when(df[i]==0,int(data)).otherwise(df[i]))

Mean value for Glucose is 121
Mean value for BloodPressure is 69
Mean value for SkinThickness is 20
Mean value for Insulin is 80
Mean value for BMI is 32


In [14]:
df.describe().show()

+-------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------------+------------------+------------------+
|summary|      Pregnancies|           Glucose|     BloodPressure|     SkinThickness|          Insulin|               BMI|DiabetesPedigreeFunction|               Age|           Outcome|
+-------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------------+------------------+------------------+
|  count|             2000|              2000|              2000|              2000|             2000|              2000|                    2000|              2000|              2000|
|   mean|           3.7035|           121.969|           72.2505|            26.665|          118.494|32.640999999999984|     0.47092999999999974|           33.0905|             0.342|
| stddev|3.306063032730656|30.533214334373536|11.970354817098098|10.0542189

In [15]:
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|
|          0|    145|           69|           20|     80|44.2|                    0.63| 31|      1|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|
|          0|    173|           78|           32|    265|46.5|                   1.159| 58|      0|
|          4|     99|           72|           17|     80|25.6|                   0.294| 28|      0|


In [16]:
for col in df.columns:
  print("correlation to outcome for {} is {}".format(col,df.stat.corr('Outcome',col)))

correlation to outcome for Pregnancies is 0.22443699263363961
correlation to outcome for Glucose is 0.48796646527321064
correlation to outcome for BloodPressure is 0.17171333286446713
correlation to outcome for SkinThickness is 0.1659010662889893
correlation to outcome for Insulin is 0.1711763270226193
correlation to outcome for BMI is 0.2827927569760082
correlation to outcome for DiabetesPedigreeFunction is 0.1554590791569403
correlation to outcome for Age is 0.23650924717620253
correlation to outcome for Outcome is 1.0


# Özellik Seçimi

In [17]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age'],outputCol='features')
output_data = assembler.transform(df)

In [18]:
output_data.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)
 |-- features: vector (nullable = true)



In [20]:
output_data.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|            features|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|          2|    138|           62|           35|     80|33.6|                   0.127| 47|      1|[2.0,138.0,62.0,3...|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|[0.0,84.0,82.0,31...|
|          0|    145|           69|           20|     80|44.2|                    0.63| 31|      1|[0.0,145.0,69.0,2...|
|          0|    135|           68|           42|    250|42.3|                   0.365| 24|      1|[0.0,135.0,68.0,4...|
|          1|    139|           62|           41|    480|40.7|                   0.536| 21|      0|[1.0,139.0,62.0,4...|
|          0|    173|           

# Modeli oluşturma ve Egitme

In [21]:
from pyspark.ml.classification import LogisticRegression
final_data = output_data.select('features','Outcome')

In [22]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Outcome: integer (nullable = true)



In [26]:
train , test = final_data.randomSplit([0.7,0.3], seed=2021)
models = LogisticRegression(labelCol='Outcome')
model = models.fit(train)

In [27]:
summary = model.summary

In [28]:
summary.predictions.describe().show()

+-------+------------------+------------------+
|summary|           Outcome|        prediction|
+-------+------------------+------------------+
|  count|              1407|              1407|
|   mean|0.3290689410092395|0.2508884150675195|
| stddev| 0.470042127824081|0.4336785568475718|
|    min|               0.0|               0.0|
|    max|               1.0|               1.0|
+-------+------------------+------------------+



# Degerlendirme ve Modeli Test Etme

In [29]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions = model.evaluate(test)

In [30]:
predictions.predictions.show(20)

+--------------------+-------+--------------------+--------------------+----------+
|            features|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|[0.0,67.0,76.0,20...|      0|[2.24709020489390...|[0.90439924645373...|       0.0|
|[0.0,67.0,76.0,20...|      0|[2.24709020489390...|[0.90439924645373...|       0.0|
|[0.0,73.0,69.0,20...|      0|[4.23515568092950...|[0.98572905230357...|       0.0|
|[0.0,74.0,52.0,10...|      0|[3.51417080557510...|[0.97108829331014...|       0.0|
|[0.0,78.0,88.0,29...|      0|[2.78505665415667...|[0.94186295179257...|       0.0|
|[0.0,84.0,82.0,31...|      0|[2.68323836540917...|[0.93603030307851...|       0.0|
|[0.0,93.0,60.0,20...|      0|[2.31869987549789...|[0.91041395858044...|       0.0|
|[0.0,93.0,100.0,3...|      0|[1.12470731615771...|[0.75486083094605...|       0.0|
|[0.0,95.0,64.0,39...|      0|[1.55338265215874...|[0.82540175505898...|    

In [34]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='Outcome')
evaluator.evaluate(model.transform(test))

0.8485987447088018

In [35]:
print("Test Error = %g" % (1.0 - evaluator.evaluate(model.transform(test))))

Test Error = 0.151401


In [36]:
type(predictions.predictions)

pyspark.sql.dataframe.DataFrame

In [37]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = predictions.predictions.select(['Outcome']).collect()
y_pred = predictions.predictions.select(['prediction']).collect()

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true,y_pred))



              precision    recall  f1-score   support

           0       0.76      0.91      0.83       372
           1       0.78      0.51      0.62       221

    accuracy                           0.76       593
   macro avg       0.77      0.71      0.72       593
weighted avg       0.77      0.76      0.75       593

[[340  32]
 [108 113]]


In [38]:
model.save("model")

In [39]:
from pyspark.ml.classification import LogisticRegressionModel
model = LogisticRegressionModel.load('model')